In [ ]:
import pandas as pd
import numpy as np

In [6]:
def process_chunk(chunk):
        print(f'Processing chunk with {len(chunk)} rows')
        print(chunk.columns)
        
def test_process_json_file(path_to_file, chunksize = 10 ** 6):
    with pd.read_json(path_to_file, lines=True, compression='bz2', chunksize=chunksize) as df_reader:
        for chunk in df_reader:
            process_chunk(chunk)


In [4]:
quotes_2020='data/quotes-2020.json.bz2'

In [7]:
test_process_json_file(quotes_2020, chunksize = 10000 )

Processing chunk with 10000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='object')
Processing chunk with 10000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='object')
Processing chunk with 10000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='object')
Processing chunk with 10000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='object')
Processing chunk with 10000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='object')
Processing chunk with 10000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='object')
Proc

KeyboardInterrupt: 

In [31]:
vocabulary= ['Muslim','Islam ','Veil ','Burqa','sharia','ideology',
             'Western','migrant','Immigration','Victims ','Protect',
             'Save','Violence','Men','Dangerous','extremist','attacks',
             'terrorism','misogyny','sexism','Equal wages','Gender equality',
             'equity','gender gap','femonationalism']

In [196]:
def process_chunk(chunk, occurences, vocabulary):
        print(f'Processing chunk with {len(chunk)} rows')
        print(chunk.columns)
        occurences = np.zeros(len(vocabulary))
        for index, word in enumerate(vocabulary):
            occurences[index] = np.sum(chunk['quotation'].str.contains(word,case=False)) 
        return occurences
        
def count_occurences_word(path_to_file, vocabulary, chunksize = 10 ** 4):
    occurences = np.zeros(len(vocabulary))
    with pd.read_json(path_to_file, lines=True, compression='bz2', chunksize=chunksize) as df_reader:
        for chunk in df_reader:
            occurences = occurences + process_chunk(chunk, occurences, vocabulary)
            for index, word in enumerate(vocabulary):
                print(occurences[index], 'occurences of the word' , word)
    return occurences

In [76]:
occurences = count_occurences_word(quotes_2020, vocabulary, chunksize = 10 ** 4)

Processing chunk with 10000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='object')
11.0 occurences of the word Muslim
3.0 occurences of the word Islam 
0.0 occurences of the word Veil 
0.0 occurences of the word Burqa
0.0 occurences of the word sharia
2.0 occurences of the word ideology
9.0 occurences of the word Western
6.0 occurences of the word migrant
1.0 occurences of the word Immigration
1.0 occurences of the word Victims 
10.0 occurences of the word Protect
1.0 occurences of the word Save
1.0 occurences of the word Violence
4.0 occurences of the word Men
0.0 occurences of the word Dangerous
2.0 occurences of the word extremist
20.0 occurences of the word attacks
5.0 occurences of the word terrorism
1.0 occurences of the word misogyny
1.0 occurences of the word sexism
0.0 occurences of the word Equal wages
0.0 occurences of the word Gender equality
15.0 occurences of the word equity
1.0 occurences 

In [91]:
occurences

array([6461.,  585.,   16.,    9.,   28., 1826., 3763., 5416.,  483.,
        190., 2466.,  980.,  591., 3844.,  158., 1039., 7503., 3528.,
        348.,  533.,    0.,   19., 3548.,   67.,    0.])

In [180]:
#Create a Dataframe with the 2020 data 
df_occ = pd.DataFrame(occurences, columns=['2020'], index=vocabulary)    

2020
Muslim           6461.0
Islam             585.0
Veil               16.0
Burqa               9.0
sharia             28.0
ideology         1826.0
Western          3763.0
migrant          5416.0
Immigration       483.0
Victims           190.0
Protect          2466.0
Save              980.0
Violence          591.0
Men              3844.0
Dangerous         158.0
extremist        1039.0
attacks          7503.0
terrorism        3528.0
misogyny          348.0
sexism            533.0
Equal wages         0.0
Gender equality    19.0
equity           3548.0
gender gap         67.0
femonationalism     0.0

In [199]:
df_occ['2020'].sum()

43401.0

## Retrieve quotation ID to inspect some of them

def process_chunk(chunk, occurences, vocabulary):
        print(f'Processing chunk with {len(chunk)} rows')
        print(chunk.columns)
        occurences = np.zeros(len(vocabulary))
        #indices_list = np.empty( dtype=object,shape=(len(vocabulary),1))
        indices_list = np.empty((len(vocabulary), 0)).tolist()
        print('The indices list is a sublist of sublist',indices_list)
        for index, word in enumerate(vocabulary):
            bool_answer = chunk['quotation'].str.contains(word,case=False) #boolean series with True when the string contains the word of interest
            indices = bool_answer[bool_answer].index.values #np array chunk indices for True results
            print(indices_list, 'indices before')
            indices_list[index] = indices.values
            print(indices_list, 'after')
            occurences[index] = np.sum(bool_answer) #summing True results
        return [occurences,indices]
    
def count_occurences_word(path_to_file, vocabulary, chunksize = 10 ** 4):
    occurences = np.zeros(len(vocabulary))
    chunk_Index = 0 
    with pd.read_json(path_to_file, lines=True, compression='bz2', chunksize=chunksize) as df_reader:
        for chunk in df_reader:
            proc_chunk = process_chunk(chunk, occurences, vocabulary)
            occurences = occurences + proc_chunk[0]
            for index, word in enumerate(vocabulary):
                print(occurences[index], 'occurences of the word' , word)
            chunk_Index += 1
    return occurences   

In [194]:
#count_occurences_word(quotes_2020, vocabulary, chunksize = 10 ** 4)

### Exploration with "Muslim Women" 

Manual exploration using the Quotebank UI 
Found more than 1000 quotations 
In the 10 first pages, i.e. 100 quotations we have 11 quotations that are relevant to our topic.
In this case what is relevant is the association of muslim women with their (supposed) protection need and their opposition to muslim men (two arguments of femonationalism).

"saving Muslim women from Muslim men" Charles Isherwood  
"oppression of Muslim women." Vladimir Putin. <---  
"those poor Muslim women." Unknown speaker    
"Are Muslim Women Oppressed?," Zohra Sarwari    
"Demonising Muslim women, the small group of Muslim women who wear this, it is more oppressive," Pauline Hanson  
"criminalise Muslim men in the guise of protecting Muslim women" Ravi Shankar Prasad      
"Instead of protecting Muslim women, the Bill criminalises Muslim men," Shashi Tharoor   
"people (Muslim women) are suffering" Anand Grover  
"Do Muslim Women Need Saving?," Lila Abu-lughod  
"Muslim women are particularly vulnerable," emily thornberry  
"traditional submissiveness of Muslim women" david cameron...  



In [178]:
voc = ['muslim women']
occurences_mw = count_occurences_word(quotes_2020, voc, chunksize = 10 ** 4)

Processing chunk with 10000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='object')
0.0 occurences of the word muslim women
Processing chunk with 10000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='object')
0.0 occurences of the word muslim women
Processing chunk with 10000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='object')
0.0 occurences of the word muslim women
Processing chunk with 10000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='object')
1.0 occurences of the word muslim women
Processing chunk with 10000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='object')
1.0 occure

In [179]:
occurences_mw

array([101.])

So we have 101 quotes in 2020 that contains muslim women. 
An extrapolation over the 12 years of dataset gives us ~1400 quotations with "muslim women" two-grams. 

### Women's right quotes

In [197]:
voc = ['women\'s right']
occurences_wr = count_occurences_word(quotes_2020, voc, chunksize = 10 ** 4)

Processing chunk with 10000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='object')
1.0 occurences of the word women's right
Processing chunk with 10000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='object')
1.0 occurences of the word women's right
Processing chunk with 10000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='object')
1.0 occurences of the word women's right
Processing chunk with 10000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='object')
3.0 occurences of the word women's right
Processing chunk with 10000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='object')
4.0 oc

In [198]:
occurences_wr

array([520.])

We have 520 quotes in 2020 that contains women's right. 
An extrapolation over the 12 years of dataset gives us ~6240 quotations with "women's right" two-grams. 

In [201]:
voc = ['women immigration']
occurences_wi = count_occurences_word(quotes_2020, voc, chunksize = 10 ** 4)

Processing chunk with 10000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='object')
0.0 occurences of the word women immigration
Processing chunk with 10000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='object')
0.0 occurences of the word women immigration
Processing chunk with 10000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='object')
0.0 occurences of the word women immigration
Processing chunk with 10000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='object')
0.0 occurences of the word women immigration
Processing chunk with 10000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype=

In [205]:
occurences_wi

array([0.])

In [202]:
voc = ['women victim']
occurences_wv = count_occurences_word(quotes_2020, voc, chunksize = 10 ** 4)

Processing chunk with 10000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='object')
0.0 occurences of the word women victim
Processing chunk with 10000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='object')
0.0 occurences of the word women victim
Processing chunk with 10000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='object')
0.0 occurences of the word women victim
Processing chunk with 10000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='object')
0.0 occurences of the word women victim
Processing chunk with 10000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='object')
0.0 occure

In [204]:
occurences_wv 

array([8.])

In [207]:
vocabulary2 = ('Equal opportunities','Equal rights','Equal status','Female','Feminism',
               'Gender discrimination','Sexism','Sexual harrasment','Women empowermenet',
               'Women emancipation','women\'s participation','Western women','non-western woman','equal pay')

In [208]:
occurences_voc2 = count_occurences_word(quotes_2020, vocabulary2, chunksize = 10 ** 4)

Processing chunk with 10000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='object')
0.0 occurences of the word Equal opportunities
0.0 occurences of the word Equal rights
0.0 occurences of the word Equal status
19.0 occurences of the word Female
3.0 occurences of the word Feminism
1.0 occurences of the word Gender discrimination
1.0 occurences of the word Sexism
0.0 occurences of the word Sexual harrasment
0.0 occurences of the word Women empowermenet
0.0 occurences of the word Women emancipation
0.0 occurences of the word women's participation
0.0 occurences of the word Western women
0.0 occurences of the word non-western woman
1.0 occurences of the word equal pay
Processing chunk with 10000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='object')
0.0 occurences of the word Equal opportunities
1.0 occurences of the word Equa

In [210]:
occurences_voc2.sum()

8804.0